In [29]:
import pandas as pd
import datetime
import pymystem3
import pymorphy3
import textdistance
import numpy as np
from nltk.stem import SnowballStemmer

## Извлечение дат

In [3]:
df = pd.DataFrame(columns=['date', 'weather'])

In [4]:
months = {
    "Января" : "January",
    "Февраля" : "February",
    "Марта" : "March",
    "Апреля" : "April",
    "Мая" : "May",
    "Июня" : "June",
    "Июля" : "July",
    "Августа" : "August",
    "Сентября" : "September",
    "Октября" : "October",
    "Ноября" : "November",
    "Декабря" : "December"
}

def parse_time(s : str) -> list[str]:
    for ru_month in months.keys():
        s = s.replace(ru_month, months[ru_month])
    l = list(s)
    i1 = l.index('[')
    i2 = l.index(' ', i1)
    l[i2] = ':'
    l.remove('[')
    l.remove(']')
    return "".join(l)

print(parse_time('10 Января [11 45]      '))

10 January 11:45      


In [5]:
def extract_date_weather(line : str) -> tuple[datetime.datetime, str]:
    l = line.split('"')
    date_string = parse_time(l[1])
    date = datetime.datetime.strptime(date_string, "%d %B %Y %H:%M")
    return (date, l[3])

In [6]:
weather = {"date" : [], "weather" : []}

In [7]:
with open("data/dataMeteo.txt") as data_file, open("data/dataMeteo_new.txt") as data_file_new:
    for line1, line2 in zip(data_file, data_file_new):
        for l in (line1, line2):
            t = extract_date_weather(l)
            weather['date'].append(t[0])
            weather['weather'].append(t[1])


In [8]:
weather

{'date': [datetime.datetime(2019, 4, 7, 11, 45),
  datetime.datetime(2024, 2, 12, 15, 18),
  datetime.datetime(2019, 4, 6, 10, 55),
  datetime.datetime(2024, 2, 12, 15, 18),
  datetime.datetime(2019, 4, 6, 9, 30),
  datetime.datetime(2024, 2, 12, 14, 25),
  datetime.datetime(2019, 4, 5, 13, 3),
  datetime.datetime(2024, 2, 12, 12, 49),
  datetime.datetime(2019, 4, 5, 10, 38),
  datetime.datetime(2024, 2, 12, 12, 43),
  datetime.datetime(2019, 4, 5, 9, 53),
  datetime.datetime(2024, 2, 12, 12, 25),
  datetime.datetime(2019, 4, 5, 8, 58),
  datetime.datetime(2024, 2, 12, 6, 15),
  datetime.datetime(2019, 4, 4, 14, 42),
  datetime.datetime(2024, 2, 11, 22, 55),
  datetime.datetime(2019, 4, 4, 6, 42),
  datetime.datetime(2024, 2, 11, 14, 10),
  datetime.datetime(2019, 4, 4, 6, 42),
  datetime.datetime(2024, 2, 11, 11, 40),
  datetime.datetime(2019, 4, 4, 6, 42),
  datetime.datetime(2024, 2, 11, 11, 35),
  datetime.datetime(2019, 4, 3, 13, 23),
  datetime.datetime(2024, 2, 11, 10, 40),
  da

In [9]:
df = pd.DataFrame(weather)
df

,date,weather
0,2019-04-07 11:45:00,8 апреля в горах Дагестана выше 2500 м в гора...
1,2024-02-12 15:18:00,14-15 февраля в Сыктывкаре ожидается аномально...
2,2019-04-06 10:55:00,7-9 апреля в горных районах муниципального обр...
3,2024-02-12 15:18:00,14-15 февраля в Сыктывкаре ожидается аномально...
4,2019-04-06 09:30:00,7 апреля в горах Дагестана выше 2500 м 7-9 ...
...,...,...
10375,2019-04-06 10:55:00,7-9 апреля в горных районах муниципального обр...
10376,2014-06-16 18:22:00,До конца дня 16 с сохранением ночью 17 июня в ...
10377,2019-04-06 09:30:00,7 апреля в горах Дагестана (выше 2500 м); 7-9 ...
10378,2014-06-16 18:18:00,Ночью 17 июня по северу Удмуртии на почве ожид...


## Извлечение названий регионов

In [46]:
mystem = pymystem3.Mystem()
morphy = pymorphy3.MorphAnalyzer()

In [47]:
def lemmatizeSentences(texts):
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(texts, 1000)  # Куски по 1000 предложений
    res = []
    # txtp = 1000 предложений
    for txtp in txtpart:
        # Объединяем 1000 предложений
        alltexts = ' '.join([txt + ' br ' for txt in txtp])

        words = mystem.lemmatize(alltexts)
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != '':
                if txt == 'br':
                    res.append(" ".join(doc))
                    doc = []
                else:
                    doc.append(txt)
    return res

In [48]:
df['weather_l'] = lemmatizeSentences(df['weather'])

In [49]:
df.head()

,date,weather,weather_l
0,2019-04-07 11:45:00,8 апреля в горах Дагестана выше 2500 м в гора...,8 апрель в гора дагестан высоко 2500 м в гора ...
1,2024-02-12 15:18:00,14-15 февраля в Сыктывкаре ожидается аномально...,14 - 15 февраль в сыктывкар ожидаться аномальн...
2,2019-04-06 10:55:00,7-9 апреля в горных районах муниципального обр...,7 - 9 апрель в горный район муниципальный обра...
3,2024-02-12 15:18:00,14-15 февраля в Сыктывкаре ожидается аномально...,14 - 15 февраль в сыктывкар ожидаться аномальн...
4,2019-04-06 09:30:00,7 апреля в горах Дагестана выше 2500 м 7-9 ...,7 апрель в гора дагестан высоко 2500 м 7 - 9 а...


In [50]:
df_towns = pd.read_csv('data/towns.csv')

In [51]:
df_towns.head()

,city,population,lat,lon,region_name,region_name_ao,region_iso_code,federal_district,okato,oktmo,kladr_id,fias_id,place_id
0,Алейск,29.0,52.492251,82.779361,Алтайский край,NaN,RU-ALT,Сибирский,1403000000,1703000001,2200000200000,ae716080-f27b-40b6-a555-cf8b518e849e,257210632.0
1,Барнаул,632.4,53.347997,83.779806,Алтайский край,NaN,RU-ALT,Сибирский,1401000000,1701000001,2200000100000,d13945a8-7017-46ab-b1e6-ede1e89317ad,258390190.0
2,Белокуриха,15.2,51.996152,84.983960,Алтайский край,NaN,RU-ALT,Сибирский,1404000000,1704000001,2200000300000,e4edca96-9b86-4cac-8c7f-cc93d9ba4cd1,258169961.0
3,Бийск,199.5,52.539386,85.213845,Алтайский край,NaN,RU-ALT,Сибирский,1405000000,1705000001,2200000400000,52f876f6-cb1d-4f23-a22f-b692609fc1e0,258937159.0
4,Горняк,12.3,50.997962,81.464336,Алтайский край,NaN,RU-ALT,Сибирский,1225501000,1625101001,2202700100000,094b3627-2699-4782-8492-4d82aac71958,259062368.0


In [52]:
df_regions = pd.read_csv('data/regions.csv')
df_regions.head()

,region
0,Алтайский край
1,Амурская область
2,Архангельская область
3,Астраханская область
4,Белгородская область


In [53]:
class MyStemmer :
    snowball_stemmer = SnowballStemmer("russian")
    def stem(self, s : str) -> str :
        s = self.snowball_stemmer.stem(s)
        if s[-2:] == "ск" :
            s = s[:-2]
        return s

In [63]:
my_stemmer = MyStemmer()
print(my_stemmer.stem("чувашский"), my_stemmer.stem("чувашия"))

чуваш чуваш


In [55]:
stop_words = ["автономная", "автономный", "область", "округ", "край", "республика"] 

In [66]:
def region_transform(s : str, stemmer) :
    s = s.lower()
    for w in stop_words :
        s = s.replace(w, "")
    return stemmer.stem(s.strip())

In [68]:
snowball_stemmer = SnowballStemmer("russian")

In [69]:
df_regions['region_stemmed'] = df_regions['region'].map(lambda x : region_transform(x, snowball_stemmer))
df_regions['regions_stemmed_full'] = df_regions['region'].map(lambda x : region_transform(x, my_stemmer))
df_regions.head()

,region,region_stemmed,regions_stemmed_full
0,Алтайский край,алтайск,алтай
1,Амурская область,амурск,амур
2,Архангельская область,архангельск,архангель
3,Астраханская область,астраханск,астрахан
4,Белгородская область,белгородск,белгород


In [70]:
regions_dict = df_regions['region_stemmed'].to_dict()
regions_dict = {v : k for k, v in regions_dict.items()}

In [75]:
stem = my_stemmer.stem

print(stem("чувашская"), ":" stem("чувашия"), textdistance.levenshtein(stem("чувашская"), stem("чувашия")), "\n",
    stem("чувашия"), ":", stem("чукотский"), textdistance.levenshtein(stem("чувашия"), stem("чукотский")), "\n",
    stem("чувашский"), ":", stem("чукотcкий"), textdistance.levenshtein(stem("чувашский"), stem("чукотский")), "\n",
    stem("чувашский"), ":", stem("чукотка"),textdistance.levenshtein(stem("чувашский"), stem("чукотка")), "\n",
    stem("карачаево-черкесия"), ":", stem("карачаево-черкесская")), textdistance.levenshtein(stem("карачево-черкесия"), stem("карачаево-черкесская"))


# print(textdistance.levenshtein(stem("чувашская"), stem("чувашия")), 
#       textdistance.levenshtein(stem("чувашия"), stem("чукотский")), 
#       textdistance.levenshtein(stem("чувашский"), stem("чукотский")),
#       textdistance.levenshtein(stem("чувашский"), stem("чукотка")),
#       textdistance.levenshtein(stem("карачево-черкесия"), stem("карачаево-черкесская")))

чувашская : чуваш 0 
 чувашия : чукот 3 
 чувашский : чукотcк 3 
 чувашский : чукотк 4 
 карачаево-черкесия : карачаево-черкес


(None, 1)

In [60]:
def region_match(s : str) -> int :
   s = Porter.stem(s)
   distances = [textdistance.levenshtein(Porter.stem(s), k) for k in regions_dict]
   return np.argmin(distances)
   
        

In [61]:
df_regions.iloc[region_match("чувашия")]

region            Чувашская Республика
region_stemmed                   чуваш
Name: 79, dtype: object

In [62]:
df.to_csv('data/weather.csv')